In [1]:
# ---------------------------------------importing required libraries----------------------------------------------------

# For automation and web scraping
import selenium  
# for saving the file and using hotkeys
import pyautogui 
from selenium import webdriver
# By will serve as a substitute for find_element_by_ method
from selenium.webdriver.common.by import By
# WebDriverWait and expected_conditions will delay the code execution until the required tag,link is loaded in the browser (useful for slow webpage)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# To delay the execution of the program
import time
import pandas as pd
# To open url
from urllib.request import urlopen
# To handle json files
import json


# ---------------------------------------scraping data from webpage------------------------------------------------------

try:
    driver_path = 'C:\Program Files\chromedriver.exe'
    driver = webdriver.Chrome(executable_path = driver_path)
except:
    print("Something wrong with the driver path given for web browser")
    
try:
    # Opening the webpage
    driver.get('https://www.amfiindia.com/nav-history-download')
except:
    print("Webpage address not available")
    
try:
    # Searching for the link to download nav report in text format
    nav_report = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'navhistorydownload')))
except:
    print("Either the webpage took too long to respond or the id for required element has changed")
    
try:
    # Clicking on the href link to navigate the driver to the webpage containing nav report
    nav_text_webpage = nav_report.find_element(By.TAG_NAME,'a')
    nav_text_webpage.click()
    time.sleep(2)
except:
    print("Link to navigate to NAV text report unavailable")
    
try:
    # Saving the file with desired file name (By default file is saved in downlods folder in C drive)
    pyautogui.hotkey('ctrl','s')
    time.sleep(2)
    pyautogui.write('scraped_nav') 
    pyautogui.press('enter')
    print('File saved successfully.')
except:
    print('Something went wrong while saving the file')
    
time.sleep(2)
driver.quit()
    
#--------------------------------------Parsing the text file to csv----------------------------------------------

try: 
    text_path = open(r"C:\Users\maidi\Downloads\scraped_nav.txt")
except:
    print("Couldn't locate the downloaded text file")

def texttocsv(text_path):
    try:
        read_file = pd.read_csv (text_path)
        read_file.to_csv ('test.csv',index=None)  
        global df
        df = pd.read_csv('test.csv',delimiter = ';')
    except:
        print("Failed to parse text to csv.")
        
    try:
        df.dropna(inplace = True)
        df['ISIN Div Reinvestment'].replace('-',None,inplace = True)
        df.replace('^\s+', '', regex=True, inplace = True) #front
        df.replace('\s+$', '', regex=True, inplace = True) #end
        print("Dataframe containing all silver fund names, numbers and scheme codes created successfully.(Saved in variable 'df')")
    except:
        print('Failed to implement mehods on the dataframe.')    
    
texttocsv(text_path)

#---------------------------------------Modifying the dataframe to extract url-----------------------------------

try:
    nav_name = df.copy()
    nav_name['url'] = "https://api.mfapi.in/mf/" + nav_name['Scheme Code']
    nav_name.drop(columns = ['Scheme Code','Net Asset Value','Date','ISIN Div Payout/ ISIN Growth','ISIN Div Reinvestment'], inplace = True)
    print("Dataframe with Scheme name and url extracted successfully.(Saved in variable 'nav_name')")
except:
    print("Error in modifying the dataframe fields to get a list of url.")

#--------------------------------------Extracting data from all url and saving in a dataframe----------------------

try:
    url_list = nav_name['url'].values
except:
    print("Failed to create a list of url to run.")
    
nav_values = pd.DataFrame()
    
# Only working with top 30 records as loading all takes a long time
for url in url_list[:30]:   
    
    try:    
        # Extracting data from url to dataframe
        response = urlopen(url)
        data_json = json.loads(response.read())
        date_nav = data_json['data']
        date_nav = pd.DataFrame.from_dict(date_nav)
    except:
        print("Failed to load data from url's to a dataframe.")
        
    try:
        # Modifying the dataframe as per our requirement
        date_nav.sort_values(by='date', inplace=True)
        date_nav.reset_index(inplace=True)
        date_nav.drop(columns=['index','date'], inplace=True)
        date_nav_add = date_nav.transpose()
        date_nav_add['url'] = url
        date_nav_add.set_index('url',inplace = True)
        nav_values = pd.concat([nav_values,date_nav_add])
    except:
        print("Failed to modify nav values dataframe as per requirement.")
        
#--------------------------------------Merging the 2 dataframes with fund name and values------------------------

try:
    # Merging the 2 dataframes on basis of common index as url
    nav_name.set_index('url',inplace = True)
    nav_data = pd.concat([nav_values,nav_name[:30]],axis = 1)
    nav_data.set_index('Scheme Name', inplace = True)
    print("Successfully saved fund names and nav values in excel file 'Mutual_Funds.xlsx'.")   
except:
    print("Failed to merge the 2 dataframes with fund names and nav values.")

nav_data.to_excel('Mutual_Funds.xlsx')

C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\3478376975.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = driver_path)


File saved successfully.
Dataframe containing all silver fund names, numbers and scheme codes created successfully.(Saved in variable 'df')
Dataframe with Scheme name and url extracted successfully.(Saved in variable 'nav_name')
Successfully saved fund names and nav values in excel file 'Mutual_Funds.xlsx'.


In [4]:
nav_data

,0,1,2,3,4,5,6,7,8,9,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975
Scheme Name,,,,,,,,,,,,,,,,,,,,,
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - IDCW,103.44600,115.06220,125.33590,149.46350,149.84520,153.81930,160.89970,102.99760,125.99330,142.70800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - MONTHLY IDCW,100.24810,104.28040,104.86460,106.77930,106.74290,110.02280,115.59890,100.05500,104.68250,109.54580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - Direct - Quarterly IDCW,101.75910,104.25250,103.89430,104.42280,104.15300,107.43610,112.99350,101.76650,104.43930,108.28790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - IDCW,10.08090,10.17570,102.97560,103.44100,114.77580,124.64850,147.71750,157.42160,161.15480,168.03010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - MONTHLY IDCW,10.00550,10.00550,100.05500,100.24780,104.02510,104.31390,105.55120,105.20380,108.11290,113.21410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - Quarterly IDCW,10.06650,10.14450,101.85310,101.75880,104.15010,103.47440,103.33710,102.75780,105.67880,110.76510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - monthly IDCW,10.00550,10.00550,100.05500,100.24700,104.00330,104.27140,105.41380,105.06720,107.97440,113.06970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - quarterly IDCW,10.04030,10.17810,10.26800,103.83200,104.28690,106.79730,106.10450,105.96600,105.37210,108.36770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - Regular Plan-Growth,10.26030,10.82010,137.45890,150.71030,169.47400,184.05170,218.11420,232.44200,255.77550,283.64080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Transformation

In [34]:
df = nav_data.copy()

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, Aditya Birla Sun Life Banking & PSU Debt Fund  - DIRECT - IDCW to Canara Robeco Banking and PSU Debt Fund- Regular Plan- Growth Option
Columns: 3976 entries, 0 to 3975
dtypes: object(3976)
memory usage: 933.2+ KB


In [36]:
df = df.apply(pd.to_numeric) # convert all columns of DataFrame

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, Aditya Birla Sun Life Banking & PSU Debt Fund  - DIRECT - IDCW to Canara Robeco Banking and PSU Debt Fund- Regular Plan- Growth Option
Columns: 3976 entries, 0 to 3975
dtypes: float64(3976)
memory usage: 933.2+ KB


In [39]:
df_growth = df.diff(axis = 1, periods = 1)

In [40]:
df_growth

,0,1,2,3,4,5,6,7,8,9,...,3966,3967,3968,3969,3970,3971,3972,3973,3974,3975
Scheme Name,,,,,,,,,,,,,,,,,,,,,
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - IDCW,NaN,11.6162,10.2737,24.1276,0.3817,3.9741,7.0804,-57.9021,22.9957,16.7147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - MONTHLY IDCW,NaN,4.0323,0.5842,1.9147,-0.0364,3.2799,5.5761,-15.5439,4.6275,4.8633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - Direct - Quarterly IDCW,NaN,2.4934,-0.3582,0.5285,-0.2698,3.2831,5.5574,-11.2270,2.6728,3.8486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - IDCW,NaN,0.0948,92.7999,0.4654,11.3348,9.8727,23.0690,9.7041,3.7332,6.8753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - MONTHLY IDCW,NaN,0.0000,90.0495,0.1928,3.7773,0.2888,1.2373,-0.3474,2.9091,5.1012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - Quarterly IDCW,NaN,0.0780,91.7086,-0.0943,2.3913,-0.6757,-0.1373,-0.5793,2.9210,5.0863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - monthly IDCW,NaN,0.0000,90.0495,0.1920,3.7563,0.2681,1.1424,-0.3466,2.9072,5.0953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - quarterly IDCW,NaN,0.1378,0.0899,93.5640,0.4549,2.5104,-0.6928,-0.1385,-0.5939,2.9956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aditya Birla Sun Life Banking & PSU Debt Fund - Regular Plan-Growth,NaN,0.5598,126.6388,13.2514,18.7637,14.5777,34.0625,14.3278,23.3335,27.8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## From the DataFrame, find:

- 1.Top 2 funds with highest average daily growth
- 2.Top 2 funds with highest overall growth
- 3.Top 2 funds with the highest growth in the last 5 days.

In [41]:
df_growth['average'] = df_growth.mean(axis=1)

In [62]:
df_growth

,0,1,2,3,4,5,6,7,8,9,...,3968,3969,3970,3971,3972,3973,3974,3975,average,overall
Scheme Name,,,,,,,,,,,,,,,,,,,,,
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - IDCW,NaN,11.6162,10.2737,24.1276,0.3817,3.9741,7.0804,-57.9021,22.9957,16.7147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007493,17.503893
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - MONTHLY IDCW,NaN,4.0323,0.5842,1.9147,-0.0364,3.2799,5.5761,-15.5439,4.6275,4.8633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006250,14.600250
Aditya Birla Sun Life Banking & PSU Debt Fund - Direct - Quarterly IDCW,NaN,2.4934,-0.3582,0.5285,-0.2698,3.2831,5.5574,-11.2270,2.6728,3.8486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004304,10.054804
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - IDCW,NaN,0.0948,92.7999,0.4654,11.3348,9.8727,23.0690,9.7041,3.7332,6.8753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044920,155.197320
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - MONTHLY IDCW,NaN,0.0000,90.0495,0.1928,3.7773,0.2888,1.2373,-0.3474,2.9091,5.1012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030380,102.107180
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - Quarterly IDCW,NaN,0.0780,91.7086,-0.0943,2.3913,-0.6757,-0.1373,-0.5793,2.9210,5.0863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029166,99.194866
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - monthly IDCW,NaN,0.0000,90.0495,0.1920,3.7563,0.2681,1.1424,-0.3466,2.9072,5.0953,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030455,101.964655
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - quarterly IDCW,NaN,0.1378,0.0899,93.5640,0.4549,2.5104,-0.6928,-0.1385,-0.5939,2.9956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027914,101.998014
Aditya Birla Sun Life Banking & PSU Debt Fund - Regular Plan-Growth,NaN,0.5598,126.6388,13.2514,18.7637,14.5777,34.0625,14.3278,23.3335,27.8653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.081974,283.465474


## Question 1: Top 2 funds with highest average daily growth

In [59]:
# Question 1

print("Top 2 funds with highest average daily growth are:")
df_growth.sort_values(by = 'average', ascending = False).head(2)['average']

Top 2 funds with highest average daily growth are:


Scheme Name
Axis Banking & PSU Debt Fund - Direct Plan - Growth Option     0.436583
Axis Banking & PSU Debt Fund - Regular Plan - Growth option    0.432319
Name: average, dtype: float64

In [60]:
df_growth['overall'] = df_growth.sum(axis=1)

## Question 2: Top 2 funds with highest overall growth

In [61]:
# Question 2

print("Top 2 funds with highest overall growth are:")
df_growth.sort_values(by = 'overall', ascending = False).head(2)['overall']

Top 2 funds with highest overall growth are:


Scheme Name
Axis Banking & PSU Debt Fund - Regular Plan - Growth option    1068.260119
Axis Banking & PSU Debt Fund - Direct Plan - Growth Option     1019.858783
Name: overall, dtype: float64

## Question 3: Top 2 funds with the highest growth in the last 5 days.

In [144]:
# Question 3

df2 = df_growth.copy()
df2.reset_index(inplace = True)

q3 = pd.DataFrame()

for ind in df2.index:
    print(ind)
    r = df_growth.iloc[ind:ind+1]
    r.dropna(axis = 1,inplace = True)
    r['last_5_days'] = r.iloc[0][-7:-2].sum()
    q3 = pd.concat([q3,r['last_5_days']])

q3

0
1
2
3
4
5
6
7
8


C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r.dropna(axis = 1,inplace = True)
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['last_5_days'] = r.iloc[0][-7:-2].sum()
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

9
10
11
12
13
14
15
16


C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['last_5_days'] = r.iloc[0][-7:-2].sum()
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r.dropna(axis = 1,inplace = True)
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

17
18
19
20
21
22
23
24
25


C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r.dropna(axis = 1,inplace = True)
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['last_5_days'] = r.iloc[0][-7:-2].sum()
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

26
27
28
29


C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r.dropna(axis = 1,inplace = True)
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['last_5_days'] = r.iloc[0][-7:-2].sum()
C:\Users\maidi\AppData\Local\Temp\ipykernel_9648\1357085784.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

,0
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - IDCW,5.9168
Aditya Birla Sun Life Banking & PSU Debt Fund - DIRECT - MONTHLY IDCW,10.5949
Aditya Birla Sun Life Banking & PSU Debt Fund - Direct - Quarterly IDCW,7.5903
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - IDCW,50.4931
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - MONTHLY IDCW,8.0895
Aditya Birla Sun Life Banking & PSU Debt Fund - REGULAR - Quarterly IDCW,5.1144
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - monthly IDCW,7.9687
Aditya Birla Sun Life Banking & PSU Debt Fund - retail - quarterly IDCW,5.2463
Aditya Birla Sun Life Banking & PSU Debt Fund - Regular Plan-Growth,124.2224
Aditya Birla Sun Life Banking & PSU Debt Fund - Retail Plan-Growth,186.4704


In [142]:
print("Top 2 funds with highest growth in last 5 days are:")
q3.rename(columns = {0:'last_5_days'}, inplace = True)
q3.sort_values(by = 'last_5_days', ascending = False).head(2)['last_5_days']

Top 2 funds with highest growth in last 5 days are:


Axis Banking & PSU Debt Fund - Direct Plan - Growth Option     911.0709
Axis Banking & PSU Debt Fund - Regular Plan - Growth option    871.4285
Name: last_5_days, dtype: float64